<a href="https://colab.research.google.com/github/tanakt-hub/Test/blob/main/BERT_PyTorch_binary_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

参考: https://tech.fusic.co.jp/posts/2021-04-23-bert-multi-classification/

In [1]:
# transformerのインストール
!pip install transformers

# Mecabのインストール
!apt install aptitude swig
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3

# fugashi
!pip install fugashi ipadic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 11.5 MB/s 
     |████████████████████████████████| 101 kB 12.7 MB/s 
     |████████████████████████████████| 6.6 MB 43.1 MB/s 
     |████████████████████████████████| 596 kB 46.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  aptitude-common libcgi-fast-perl libcgi-pm-perl libclass-accessor-perl
  libcwidget3v5 libencode-locale-perl libfcgi-perl libhtml-parser-perl
  libhtml-tagset-perl libhttp-date-perl libhttp-message-perl libio-html-perl
  libio-string-

In [2]:
import os
import urllib.request
import re
import csv
import tarfile
import torch
import numpy as np
import pandas as pd

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [3]:
#ライブドアコーパス(https://www.rondhuit.com/download.html#ldcc)の展開
!wget https://www.rondhuit.com/download/ldcc-20140209.tar.gz
!tar zxf ldcc-20140209.tar.gz

--2022-07-31 22:37:35--  https://www.rondhuit.com/download/ldcc-20140209.tar.gz
Resolving www.rondhuit.com (www.rondhuit.com)... 59.106.19.174
Connecting to www.rondhuit.com (www.rondhuit.com)|59.106.19.174|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8855190 (8.4M) [application/x-gzip]
Saving to: ‘ldcc-20140209.tar.gz’

ldcc-20140209.tar.g 100%[===================>]   8.44M  4.34MB/s    in 1.9s    

2022-07-31 22:37:37 (4.34 MB/s) - ‘ldcc-20140209.tar.gz’ saved [8855190/8855190]



In [4]:
#サンプル文書の追加
import urllib.request
SampleDoc = "https://raw.githubusercontent.com/tanakt-hub/Test/main/data/SampleDoc.txt"

res = urllib.request.urlopen(SampleDoc)
res = res.read().decode("utf-8")
SplitDoc = res.replace('\n', '').split('。')


In [5]:
!mkdir /content/text/sampledoc

In [6]:
for c, i in enumerate(SplitDoc):
  f = open('/content/text/sampledoc/' + str(c) + '.txt', 'w')
  f.write(i)
  f.close()

In [7]:
# 分類したい種類の対象や数はここで調整する

fname_class_list = {
    "sampledoc": [],
    "Others": []
}
target_genres = list(fname_class_list.keys())

In [8]:
#print(target_genres)

In [9]:
def remove_brackets(inp):
    # 記号とかを除く
    brackets_tail = re.compile('【[^】]*】$')
    brackets_head = re.compile('^【[^】]*】')
    output = re.sub(brackets_head, '', re.sub(brackets_tail, '', inp))
    return output

In [10]:
#文書をDFに読み込み
import pandas as pd
import glob
import os
import re
from tqdm import tqdm_notebook as tqdm
#preprocessing
#dirlist = ["dokujo-tsushin","it-life-hack","kaden-channel","livedoor-homme", "movie-enter","peachy","smax","sports-watch","topic-news", "sampledoc"]
dirlist = ["livedoor-homme", "sampledoc"]
df = pd.DataFrame(columns=['media_name', 'label', 'sentence'])
for i in tqdm(dirlist):
    path = "/content/text/" + i + "/*.txt"
    files = glob.glob(path)
    files.pop()
    for j in tqdm(files):
        f = open(j)
        data = f.read() 
        f.close()
        if i == "sampledoc":
          t = pd.Series([i,0,remove_brackets(data)],index = df.columns)
          df  = df.append(t,ignore_index=True)          
        if i != "sampledoc":
          tmp = re.split("[\n。？]",data)[3:]
          for s in tmp:
            if len(s) > 10:
              t = pd.Series(['Others',1,remove_brackets(s)],index = df.columns)
              df  = df.append(t,ignore_index=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # This is added back by InteractiveShellApp.init_path()


  0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  from ipykernel import kernelapp as app


  0%|          | 0/511 [00:00<?, ?it/s]

  0%|          | 0/2455 [00:00<?, ?it/s]

In [11]:
df = df.dropna(how='any') # nanのところは落とす

print(f'データサイズ： {df.shape}')
display(df.sample(20))

データサイズ： (21106, 3)


,media_name,label,sentence
11523,Others,1,このBRZは「Pure Handling Delight−新しい次元の運転する愉しさ−」をコ...
3560,Others,1,"W：読者モデルが有名になったり、ちょいワルだったり、不景気もあったりで、ファッションが""モテ..."
13213,Others,1,・vol.3 - （ゴルフに夢中な男子諸君、女子にモテたいならコレを読め）
18395,Others,1,「BeeTV」は月額315円で全作品が見放題
8485,Others,1,“たかがアイドル”だからこそできることがある———秋元
15136,Others,1,・生徒・教授ともに世界のトップクラスが集まる環境下で切磋琢磨することで、グローバルな...
3582,Others,1,・前倒しが進むセール時期—消費者は喜ぶも、、、業界を混乱させる“負のスパイラル”の要因とは
10792,Others,1,この際に排出されるのは水のみという、とてもクリーンなシステムなのだ
14507,Others,1,収入によって変わってくるかとは思いますが、「年収の10％」が一つの目安です
5204,Others,1,24歳 男性 <悩める相談者 No.05>


In [12]:
# モデルに飲ませるデータと、ラベルを準備
sentences = df.sentence.values
labels = df.label.values

# トークナイズする
from transformers import BertJapaneseTokenizer
tokenizer = BertJapaneseTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')

## 確認
print(' Original: ', sentences[0])
print('Tokenized: ', tokenizer.tokenize(sentences[0]))
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sentences[0])))


# 最大トークン数の確認
max_len = 0
# 1文づつ処理
for sentence in sentences:
    # Tokenizeで分割
    token_words = tokenizer.tokenize(sentence)
    # 文章数を取得してリストへ格納
    if len(token_words) > max_len:
        max_len = len(token_words)
# 最大の値を確認
print('最大単語数: ', max_len)
print('上記の最大単語数にSpecial token（[CLS], [SEP]）の+2をした値が最大単語数')



 Original:  　〜「判断」「指示する」ということ〜｜実践！問題解決力テスト【第1問】 
Tokenized:  ['〜', '「', '判断', '」', '「', '指示', 'する', '」', 'という', 'こと', '〜', '|', '実践', '!', '問題', '解決', '力', 'テスト', '【', '第', '1', '問', '】']
Token IDs:  [1143, 36, 2827, 38, 36, 4211, 34, 38, 140, 45, 1143, 7686, 7159, 679, 749, 3659, 691, 4609, 9680, 97, 17, 595, 9594]
最大単語数:  249
上記の最大単語数にSpecial token（[CLS], [SEP]）の+2をした値が最大単語数


In [13]:
#import numpy
#print(len(labels))
#print(sum(labels))
#print(len(labels)-sum(labels))
#print(type(labels))
#numpy.set_printoptions(threshold=numpy.inf)

#print(labels)
#tns = torch.tensor(labels.astype(int))
#print(tns)

In [14]:
import numpy as np

input_ids = []
attention_masks = []

# 1文づつ処理
for sentence in sentences:
    encoded_dict = tokenizer.encode_plus(
                        sentence,                      
                        add_special_tokens = True, # Special Tokenの追加
                        max_length = max_len + 2,           # 文章の長さを固定（Padding/Trancatinating）
                        pad_to_max_length = True,# PADDINGで埋める
                        return_attention_mask = True,   # Attention maksの作成
                        return_tensors = 'pt',     #  Pytorch tensorsで返す
                   )

    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])

# リストに入ったtensorを縦方向（dim=0）へ結合
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)

# tenosor型に変換
labels = torch.tensor(labels.astype(int))

# 確認
print('Original: ', sentences[0])
print('Token IDs:', input_ids[0])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2329: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Original:  　〜「判断」「指示する」ということ〜｜実践！問題解決力テスト【第1問】 
Token IDs: tensor([   2, 1143,   36, 2827,   38,   36, 4211,   34,   38,  140,   45, 1143,
        7686, 7159,  679,  749, 3659,  691, 4609, 9680,   97,   17,  595, 9594,
           3,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,

In [15]:
from torch.utils.data import TensorDataset, random_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

dataset = TensorDataset(input_ids, attention_masks, labels)

# 90%取得
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size

# データセットを分割
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('訓練データ数：{}'.format(train_size))
print('検証データ数: {} '.format(val_size))

# データローダーの作成
batch_size = 16

# 訓練データローダー
train_dataloader = DataLoader(
    train_dataset,  
    sampler = RandomSampler(train_dataset), # ランダムにデータを取得してバッチ化
    batch_size = batch_size
)

# 検証データローダー
validation_dataloader = DataLoader(
    val_dataset, 
    sampler = SequentialSampler(val_dataset), # 順番にデータを取得してバッチ化
    batch_size = batch_size
)


from transformers import BertForSequenceClassification, AdamW, BertConfig

# BertForSequenceClassification 学習済みモデルのロード
model = BertForSequenceClassification.from_pretrained(
    "cl-tohoku/bert-base-japanese-whole-word-masking", # 日本語Pre trainedモデルの指定
    num_labels = len(target_genres), # ここもよしなに反映される
    output_attentions = False, # アテンションベクトルを出力するか
    output_hidden_states = False, # 隠れ層を出力するか
)

# モデルをGPUへ転送
model.cuda()

訓練データ数：18995
検証データ数: 2111 


Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialize

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [16]:
from torch.nn import functional as F

# 最適化手法の設定
optimizer = AdamW(model.parameters(), lr=1e-5)

# 訓練パートの定義
def train(model):
    model.train() # 訓練モードで実行
    train_loss = 0
    for batch in train_dataloader:# train_dataloaderはword_id, mask, labelを出力する点に注意
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        optimizer.zero_grad()
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)
        loss = F.cross_entropy(outputs.logits, b_labels)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        train_loss += loss.item()
    return train_loss

# テストパートの定義
def validation(model):
    model.eval()# 訓練モードをオフ
    val_loss = 0
    with torch.no_grad(): # 勾配を計算しない
        for batch in validation_dataloader:
            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)
            b_labels = batch[2].to(device)
            with torch.no_grad():        
                outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)
                loss = F.cross_entropy(outputs.logits, b_labels)
            val_loss += loss.item()
    return val_loss


# 学習の実行
max_epoch = 4
train_loss_ = []
test_loss_ = []

for epoch in range(max_epoch):
    train_ = train(model)
    test_ = validation(model)
    print(f'epoch {epoch} loss : {test_}')
    train_loss_.append(train_)
    test_loss_.append(test_)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


epoch 0 loss : 0.7199541515074088
epoch 1 loss : 1.082644888341747
epoch 2 loss : 0.09749193740617557
epoch 3 loss : 0.22201304088457618


In [18]:
model.eval()# 訓練モードをオフ

cnt1 =0 
cnt2 =0
try_count = 0
correct_count = 0
for batch in validation_dataloader:
    b_input_ids = batch[0].to(device)
    b_input_mask = batch[1].to(device)
    b_labels = batch[2].to(device)
    with torch.no_grad():   
        # 学習済みモデルによる予測結果をpredsで取得     
        preds = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        #logits_df = pd.DataFrame(preds[0].cpu().numpy(), columns=['logit_0', 'logit_1', 'logit_2', 'logit_3', 'logit_4', 'logit_5', 'logit_6', 'logit_7', 'logit_8'])
        logits_df = pd.DataFrame(preds[0].cpu().numpy(), columns=['logit_0', 'logit_1'])
        pred_df = pd.DataFrame(np.argmax(preds[0].cpu().numpy(), axis=1), columns=['pred_label'])
        label_df = pd.DataFrame(b_labels.cpu().numpy(), columns=['true_label'])
        result_df = pd.concat([logits_df, pred_df, label_df], axis=1)
        df_bool = result_df['pred_label'] == result_df['true_label']
        try_count += df_bool.shape[0]
        correct_count += sum(df_bool)
        
    #cnt1+=1
    #cnt2 = len(result_df)
    #print(f'{cnt1}:{cnt2}')
print(f'accuracy = {correct_count/try_count}')
print(f'{correct_count}:{try_count}')

accuracy = 0.9995262908574135
2110:2111


In [19]:
## 予測結果の確認
print(f'出力:{preds}')

出力:SequenceClassifierOutput(loss=None, logits=tensor([[-5.4695,  6.2579],
        [-5.4148,  6.1947],
        [-5.6774,  6.4365],
        [-5.5185,  6.3254],
        [-5.6440,  6.4091],
        [-5.5481,  6.2427],
        [-5.4353,  6.1242],
        [-5.6727,  6.4169],
        [-5.3672,  6.1206],
        [ 5.0253, -5.7501],
        [-5.6375,  6.4509],
        [-5.6759,  6.4201],
        [-5.6094,  6.3957],
        [-5.2474,  5.9698],
        [-5.6331,  6.3498]], device='cuda:0'), hidden_states=None, attentions=None)


In [21]:
# 比較しやすい様にpd.dataframeへ整形
import pandas as pd
import numpy as np
# pd.dataframeへ変換（GPUに乗っているTensorはgpu->cpu->numpy->dataframeと変換）
#logits_df = pd.DataFrame(preds[0].cpu().numpy(), columns=['logit_0', 'logit_1', 'logit_2', 'logit_3', 'logit_4', 'logit_5', 'logit_6', 'logit_7', 'logit_8'])
logits_df = pd.DataFrame(preds[0].cpu().numpy(), columns=['logit_0', 'logit_1'])
## np.argmaxで大き方の値を取得
pred_df = pd.DataFrame(np.argmax(preds[0].cpu().numpy(), axis=1), columns=['pred_label'])
label_df = pd.DataFrame(b_labels.cpu().numpy(), columns=['true_label'])

accuracy_df = pd.concat([logits_df, pred_df, label_df], axis=1)

print(accuracy_df)

     logit_0   logit_1  pred_label  true_label
0  -5.469470  6.257926           1           1
1  -5.414754  6.194726           1           1
2  -5.677351  6.436471           1           1
3  -5.518544  6.325449           1           1
4  -5.643960  6.409081           1           1
5  -5.548134  6.242715           1           1
6  -5.435328  6.124215           1           1
7  -5.672715  6.416859           1           1
8  -5.367240  6.120552           1           1
9   5.025278 -5.750068           0           0
10 -5.637507  6.450896           1           1
11 -5.675897  6.420127           1           1
12 -5.609402  6.395732           1           1
13 -5.247427  5.969838           1           1
14 -5.633068  6.349826           1           1
